In [ ]:
library(magrittr)

library(corrplot)

library(preprocessCore)

library(DNAcopy)

library(RColorBrewer)

library(reshape)

library(ggplot2)

In [ ]:
##vecterized function to calculate nucleosome wrapping score
getnrs <- function(dataframe){
    df <- dataframe
    matrix <- as.matrix(df[,-(1:3)])
    nrow <- nrow(matrix)
    ncol <- ncol(matrix)
    per <- round(nrow/100)
    nrs <- matrix[,-ncol]
    for (i in 1:nrow){  ## for each bin, calculate NRS for each bkp
        nrsi <- rep(NaN, ncol-1)
        rowi <- matrix[i,]
        sumi <- sum(matrix[i,])
        for (j in 1:(ncol-1)){
            nrsi[j] <- (sum(rowi[(j+1):ncol])-sum(rowi[1:j]))/sumi
            }
        nrs[i,] <- nrsi
        
        ###print progression
        if((i%%per) == 0 ){
            message(paste0(round(i/nrow * 100), '% completed'))
        }
    }
    nrs <- as.data.frame(cbind(df[,1:3],nrs)) 
    return(nrs)
}

In [ ]:
genome <- "mm10"

In [ ]:
genome_file <- "/data/zqwen/genome/mm10/mm10.chrom.filter.sort.sizes"

In [ ]:
bin <- "100kb"

In [ ]:
binsize <- 100000

In [ ]:
histone <- "1-R1-crosslink-MNCUT-30min-1" ##sample prefix

In [ ]:
hist.dir <- paste0("/data/zqwen/subnucleosome/revision/06.nri/")

In [ ]:
countfile <- paste0(hist.dir, histone, ".", genome, ".", bin, ".raw.counts.txt")

In [ ]:
rawcounts <- read.delim(countfile, header = TRUE)

In [ ]:
names(rawcounts)[1:3] <- c("Chr", "Start", "End")

In [ ]:
bincounts <- rowSums(rawcounts[,4:ncol(rawcounts)])

In [ ]:
index <- (bincounts > 0)

In [ ]:
rawcounts <- rawcounts[index,]

### Calculate nrs for each bin

In [ ]:
system.time(nrs <- getnrs(rawcounts))

### Register the RAW nrs80-nrs160 data into bedGraph files

In [ ]:
for (i in 9:17){
    
    message("Writing raw: ", names(nrs)[i])
    
    rawnrs <- nrs[,c(1:3, i)]
    
    filename <- paste0(bin.dir,  histone, ".mm10.", bin, ".", names(nrs)[i],".bdg") 
        
    write.table(rawnrs, filename, sep= "\t", row.names=FALSE, quote=FALSE, col.names = FALSE)
    
}

## z-score transform of nrs80-nrs160

In [ ]:
nri <- scale(nrs[,9:17])

In [ ]:
nri <- cbind(nrs[,1:3], nri)

In [ ]:
names(nri) <- gsub("nrs","nri", names(nri), fixed = T)

### Register the z-score nrs nri80-nri160 data into bedGraph files

In [ ]:
for (i in 4:12){
    
    message("Writing nri: ", names(nri)[i])
    
    rawnrs <- nri[,c(1:3, i)]
    
    filename <- paste0(bin.dir,  histone, ".mm10.", bin, ".", names(nri)[i],".bdg")
        
    write.table(rawnrs, filename, sep= "\t", row.names=FALSE, quote=FALSE, col.names = FALSE)
    
}

### Loess smooth nri140

In [ ]:
nri140 <- nri[,c("Chr", "Start", "End", "nri140bp")]

In [ ]:
chrs <- unique(nri140$Chr)

In [ ]:
binnum <- 10

In [ ]:
Loess=data.frame()

for(Chr in chrs){
    message("Current chrom: " , Chr)
    subnrs <- nri140[nri140$Chr == Chr,]
    lspan=binnum*binsize/(max(subnrs$Start)-min(subnrs$Start)) 
    nrsfit=loess(subnrs[,4] ~ subnrs$Start, span=lspan, surface = c("direct"))
    nrilsm <- data.frame(c(rep(Chr,times=nrsfit$n)), nrsfit$x, subnrs[which(subnrs$Start %in% nrsfit$x), 3], nrsfit$fitted)
    if(nrow(Loess)!=0){ Loess = rbind(Loess,nrilsm)}
    if(nrow(Loess)==0){ Loess = nrilsm}
    }

names(Loess)=c("Chr" , "Start" , "End" , "Fitted")

In [ ]:
bdgname <- paste0(bin.dir,  histone, ".mm10.", bin, ".", "nri140bp.", binnum, "bins.smoothed.bdg")

In [ ]:
write.table(Loess, bdgname, sep= "\t", row.names=FALSE, quote=FALSE, col.names = FALSE)

In [ ]:
bwname <- paste0(bin.dir,  histone, ".mm10.", bin, ".", "nri140bp.", binnum, "bins.smoothed.bw")

In [ ]:
system2("bedGraphToBigWig", paste(bdgname, genome_file, bwname, sep=" "), wait = TRUE, stderr = TRUE, stdout = TRUE)